#### Loading saved data from joblib

In [39]:
# To get the folder where the script is running
import os

print("Current directory:", os.getcwd())

folder = r'C:/Users/FFFO CASHIER PT/OneDrive\Desktop/Mini Capstone project/database'
print("Files in target folder:", os.listdir(folder))


Current directory: c:\Users\FFFO CASHIER PT\OneDrive\Desktop\Mini capstone project\Notebook
Files in target folder: ['dune_data.pkl', 'wrapped-bitcoin_prices.pkl']


In [40]:
import os
import joblib

# Load price data from joblib file
df_price = joblib.load(r'C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Mini Capstone project/database/wrapped-bitcoin_prices.pkl')

# Load whale transfer data from joblib file
df_whale = joblib.load(r'C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Mini Capstone project/database/dune_data.pkl')


### Load & Prepare Price Data (df_price)

In [ ]:
# Check column names
print(df_price.columns)

Index(['timestamp', 'price'], dtype='object')


In [ ]:
print(df_price.index)

RangeIndex(start=0, stop=91, step=1)


In [ ]:
df_price.index.name = 'timestamp'

In [ ]:
# Convert timestamp to datetime 
df_price['timestamp'] = pd.to_datetime(df_price['timestamp'])

# Set timestamp as index
df_price.set_index('timestamp', inplace=True)

# Now set daily frequency and forward fill missing days
df_price = df_price.asfreq('D', method='ffill')

# Reset index if you want 'timestamp' back as a column
df_price = df_price.reset_index()

# Create 'date' column from timestamp for merging
df_price['date'] = df_price['timestamp'].dt.floor('D')


### Load & Prepare Whale Data (df_whale)

In [ ]:
import joblib
import pandas as pd

# Load the raw whale data (likely a list of dicts)
rows = joblib.load(r'C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Mini Capstone project/database/dune_data.pkl')

# Convert it into a DataFrame
df_whale = pd.DataFrame(rows)

# Preview
print(df_whale.head())

                                     receiver  \
0  0x9a62db4c17146172c0b88e7e439df169a0f93e0e   
1  0xa4b9569bf942c3aad23c0c2d322fe4aff8e1bf30   
2  0x2a49eae5cca3f050ebec729cf90cc910fadaf7a2   
3  0xd49a3ff72739e3fe9537645acad3ba3e65f6690d   
4  0x51c72848c68a965f66fa7a88855f9f7784502a7f   

                                       sender                         time  \
0  0xa3a7b6f88361f48403514059f1f16c8e78d60eec  2025-08-06 18:56:11.000 UTC   
1  0xd49a3ff72739e3fe9537645acad3ba3e65f6690d  2025-08-06 18:51:35.000 UTC   
2  0xfa8c996e158b80d77fbd0082bb437556a65b96e0  2025-08-06 18:32:23.000 UTC   
3  0x39c1cc6e689f001567f80b279277f921ce88e6a5  2025-08-06 18:29:11.000 UTC   
4  0xe8f7c89c5efa061e340f2d2f206ec78fd8f7e124  2025-08-06 18:28:11.000 UTC   

      usd_value  
0  1.557364e+05  
1  2.450904e+06  
2  1.225749e+05  
3  2.454708e+06  
4  1.368616e+05  


In [ ]:
# Each transaction is counted once for sender and once for receiver
# So this way the whale_volume and whale_count wont be assumed to be 0

df_sender = df_whale[['time', 'usd_value']].copy()
df_sender['role'] = 'sender'

df_receiver = df_whale[['time', 'usd_value']].copy()
df_receiver['role'] = 'receiver'

# Combine both
df_whale_dual = pd.concat([df_sender, df_receiver], ignore_index=True)



In [ ]:
print(df_whale_dual['time'].dtype)


datetime64[ns]


In [ ]:
# Time is already in datetime, convert it directly

df_whale_dual['time'] = pd.to_datetime(df_whale_dual['time'])

In [ ]:
# Floor time to daily to match price granularity
df_whale_dual['date'] = df_whale_dual['time'].dt.floor('D')

# Group by date
whale_daily = df_whale_dual.groupby('date').agg(
    whale_volume=('usd_value', 'sum'),
    whale_count=('usd_value', 'count')  # counts both senders and receivers
).reset_index()

# Rename columns
whale_daily.columns = ['date', 'whale_volume', 'whale_count']


### Merging Price movement and Whale Activity

In [ ]:
# Checking for the unique dates in both dataframes

print(df_price['date'].min(), "to", df_price['date'].max())
print(whale_daily['date'].min(), "to", whale_daily['date'].max())


2025-05-09 00:00:00 to 2025-08-06 00:00:00
2025-06-07 00:00:00 to 2025-08-06 00:00:00


In [ ]:
# Confirming the data types of date columns

print(df_price['date'].dtype)
print(whale_daily['date'].dtype)


datetime64[ns]
datetime64[ns]


In [ ]:
# Preview the whale data before merge to check if values are non-zero

print(whale_daily.head())
print(whale_daily.describe())


        date  whale_volume  whale_count
0 2025-06-07  3.447736e+08          114
1 2025-06-08  9.437152e+08          894
2 2025-06-09  3.807983e+09         2752
3 2025-06-10  2.099090e+09         2562
4 2025-06-11  1.847041e+09         1958
                      date  whale_volume  whale_count
count                   61  6.100000e+01    61.000000
mean   2025-07-07 00:00:00  2.992789e+09  1978.295082
min    2025-06-07 00:00:00  3.447736e+08   114.000000
25%    2025-06-22 00:00:00  1.673295e+09  1332.000000
50%    2025-07-07 00:00:00  2.348700e+09  1958.000000
75%    2025-07-22 00:00:00  3.501512e+09  2562.000000
max    2025-08-06 00:00:00  1.288021e+10  3804.000000
std                    NaN  2.224794e+09   807.369460


In [57]:
# Merge price and whale daily data on 'date'
df_merged = pd.merge(df_price, whale_daily, on='date', how='left')

# Fill NaN values in whale data with 0 (for days without whale activity)
df_merged['whale_volume'] = df_merged['whale_volume'].fillna(0)
df_merged['whale_count'] = df_merged['whale_count'].fillna(0)

# Final structure
print(df_merged.tail())


    timestamp          price       date  whale_volume  whale_count
85 2025-08-02  113251.979613 2025-08-02  1.074155e+10       1800.0
86 2025-08-03  112513.216187 2025-08-03  4.898928e+09       1048.0
87 2025-08-04  114185.282407 2025-08-04  5.091967e+09       1650.0
88 2025-08-05  114923.152934 2025-08-05  5.274239e+09       1740.0
89 2025-08-06  114106.509533 2025-08-06  3.175733e+09        996.0


In [58]:
# Calculate price percentage change
df_merged = df_merged.sort_values('date')
df_merged['price_change_pct'] = df_merged['price'].pct_change() * 100

# Rename columns
df_merged.rename(columns={
    'date': 'datetime',
    'whale_count': 'whale_tx_count',
    'whale_volume': 'whale_tx_volume'
}, inplace=True)

# Select columns
final_df = df_merged[['datetime', 'price', 'price_change_pct', 'whale_tx_count', 'whale_tx_volume']]

print(final_df.tail())


     datetime          price  price_change_pct  whale_tx_count  \
85 2025-08-02  113251.979613         -2.050064          1800.0   
86 2025-08-03  112513.216187         -0.652318          1048.0   
87 2025-08-04  114185.282407          1.486107          1650.0   
88 2025-08-05  114923.152934          0.646205          1740.0   
89 2025-08-06  114106.509533         -0.710600           996.0   

    whale_tx_volume  
85     1.074155e+10  
86     4.898928e+09  
87     5.091967e+09  
88     5.274239e+09  
89     3.175733e+09  
